# Scrape betting lines from ESPN

This notebook will be later adapted into a script and integrated into Geiger's Excel spreadsheet for NFL picks. Ideally, it will be as intuitive as Mike's idea of "press a button and it updates lines".

First, we explore what Fanduel site looks like in `bs4`.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)

In [2]:
import os
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
os.chdir('drive/MyDrive/Colab Notebooks/NFL Bet Line/')
!ls

'2021-2022 NFL PickEm.gsheet'  'Scrape betting lines from ESPN.ipynb'
 nfl_teams.csv


## What info do we need for the spreadsheet?

https://www.espn.com/nfl/lines

- The two numbers listed under "LINE".
    - Take greater of two numbers, attach it to "TotalPts", other is "Spread"
    - Also pull team names attached to greater #

In [4]:
url = 'https://www.espn.com/nfl/lines'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
parsed = soup.find_all('table')
parsed[0]

<table class="Table Table--align-right" style="border-collapse:collapse;border-spacing:0"><thead class="Table__header-group Table__THEAD"><tr class="Table__sub-header Table__TR Table__even"><th class="Table__TH" title="">1:00 PM</th><th class="Table__TH" title="">REC (ATS)</th><th class="Table__TH" title="">LINE</th><th class="Table__TH" title="">OPEN</th><th class="Table__TH" title="">ML</th><th class="Table__TH" title="">FPI</th></tr></thead><tbody class="Table__TBODY"><tr class="Table__TR Table__TR--sm Table__even" data-idx="0"><td class="Table__TD"><div class="flex items-start mr7"><span class="pr3 TeamLink__Logo"><a class="AnchorLink" href="/nfl/team/_/name/cle/cleveland-browns" tabindex="0"></a></span><a class="AnchorLink" data-clubhouse-uid="s:20~l:28~t:5" href="/nfl/team/_/name/cle/cleveland-browns" tabindex="0">Cleveland Browns</a></div></td><td class="Table__TD">4-4 (4-4-0)</td><td class="Table__TD">47.5</td><td class="Table__TD">45.5</td><td class="Table__TD">+115</td><td cl

In [5]:
home = []
away = []
total = []
spread = []

for game in parsed:
    try:
        # find team names
        home_name = game.find_all('td')[0].img.get('title')
        away_name = game.find_all('td')[6].img.get('title')
        # get total pts and/or spread
        # the order is inconsistent here
        # line is always relative to favored team
        totalpts = float(game.find_all('td')[2].get_text())
        line = float(game.find_all('td')[8].get_text())
        home.append(home_name)
        away.append(away_name)
        total.append(totalpts)
        spread.append(line)
    except:
        pass

week = pd.DataFrame(list(zip(home, away, total, spread)),
                    columns=['Home', 'Away', 'Total', 'Spread'])
week

,Home,Away,Total,Spread
0,Cleveland Browns,Cincinnati Bengals,47.5,-2.5
1,Denver Broncos,Dallas Cowboys,49.5,-10.0
2,Houston Texans,Miami Dolphins,46.5,-5.5
3,Atlanta Falcons,New Orleans Saints,41.5,-6.0
4,Las Vegas Raiders,New York Giants,-3.0,46.5
5,New England Patriots,Carolina Panthers,-3.5,41.5
6,Buffalo Bills,Jacksonville Jaguars,-14.5,48.5
7,Minnesota Vikings,Baltimore Ravens,50.5,-6.0
8,Los Angeles Chargers,Philadelphia Eagles,-1.5,49.5
9,Green Bay Packers,Kansas City Chiefs,48.0,-7.0


In [6]:
to_switch = week.loc[(week['Total'] < 0)].copy()
to_switch['TempTotal'] = to_switch['Total']
to_switch

,Home,Away,Total,Spread,TempTotal
4,Las Vegas Raiders,New York Giants,-3.0,46.5,-3.0
5,New England Patriots,Carolina Panthers,-3.5,41.5,-3.5
6,Buffalo Bills,Jacksonville Jaguars,-14.5,48.5,-14.5
8,Los Angeles Chargers,Philadelphia Eagles,-1.5,49.5,-1.5


In [7]:
to_switch['Total'] = to_switch['Spread']
to_switch['Spread'] = abs(to_switch['TempTotal'])
to_switch

,Home,Away,Total,Spread,TempTotal
4,Las Vegas Raiders,New York Giants,46.5,3.0,-3.0
5,New England Patriots,Carolina Panthers,41.5,3.5,-3.5
6,Buffalo Bills,Jacksonville Jaguars,48.5,14.5,-14.5
8,Los Angeles Chargers,Philadelphia Eagles,49.5,1.5,-1.5


In [8]:
week.update(to_switch)

In [9]:
week

,Home,Away,Total,Spread
0,Cleveland Browns,Cincinnati Bengals,47.5,-2.5
1,Denver Broncos,Dallas Cowboys,49.5,-10.0
2,Houston Texans,Miami Dolphins,46.5,-5.5
3,Atlanta Falcons,New Orleans Saints,41.5,-6.0
4,Las Vegas Raiders,New York Giants,46.5,3.0
5,New England Patriots,Carolina Panthers,41.5,3.5
6,Buffalo Bills,Jacksonville Jaguars,48.5,14.5
7,Minnesota Vikings,Baltimore Ravens,50.5,-6.0
8,Los Angeles Chargers,Philadelphia Eagles,49.5,1.5
9,Green Bay Packers,Kansas City Chiefs,48.0,-7.0


In [10]:
nfl_abbr = pd.read_csv('nfl_teams.csv')
nfl_abbr = nfl_abbr.drop(['ID', 'Conference', 'Division'], axis=1)

week_abbr_temp = week.merge(nfl_abbr, left_on='Home', right_on='Name')
week_abbr_temp = week_abbr_temp.rename(columns={'Abbreviation':'Home Abbrv'})

week_abbr = week_abbr_temp.merge(nfl_abbr, left_on='Away', right_on='Name')
week_final = week_abbr.drop(['Name_x','Name_y', 'Home', 'Away'], axis=1)
week_final = week_final.rename(columns={'Home Abbrv':'Home','Abbreviation':'Away'})

In [11]:
week_final = week_final[['Home','Away','Total','Spread']]
week_final

,Home,Away,Total,Spread
0,CLE,CIN,47.5,-2.5
1,DEN,DAL,49.5,-10.0
2,HOU,MIA,46.5,-5.5
3,ATL,NO,41.5,-6.0
4,LV,NYG,46.5,3.0
5,NE,CAR,41.5,3.5
6,BUF,JAX,48.5,14.5
7,MIN,BAL,50.5,-6.0
8,LAC,PHI,49.5,1.5
9,GB,KC,48.0,-7.0


# Hell yeah!

We got it and in a cool and easy to digest and digestible and easy to digest format! Let's turn that into a script later. For now, let's do things with the sheets snippet example and pray for the best.

In [12]:
!pip install gspread==4.0.1

  Attempting uninstall: gspread
    Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [13]:
from google.colab import auth
auth.authenticate_user()

import gspread
print(gspread.__version__)
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

4.0.1


In [14]:
spsh = gc.open('2021-2022 NFL PickEm')

# always comment out the line below, don't risk overwriting past weeks on accident
ws = spsh.worksheet('Week 9')

In [15]:
ws

<Worksheet 'Week 9' id:1049595123>

In [16]:
team_col = 2
tot_sprd_col = 6 

for h, a, tot, sprd in zip(week_final['Home'], week_final['Away'], week_final['Total'], week_final['Spread']):
  ## update teams
  # updates home team in all of row 2
  ws.update_cell(row=2, col=team_col, value=h)
  # updates away team in all of row 3
  ws.update_cell(row=3, col=team_col, value=a)
  # shift 8 rows to the right for next team
  team_col += 8

  # update total and spread
  ws.update_cell(row=2, col=tot_sprd_col, value=tot)
  ws.update_cell(row=3, col=tot_sprd_col, value=sprd)
  tot_sprd_col += 8
  print(h, a, tot, sprd)

CLE CIN 47.5 -2.5
DEN DAL 49.5 -10.0
HOU MIA 46.5 -5.5
ATL NO 41.5 -6.0
LV NYG 46.5 3.0
NE CAR 41.5 3.5
BUF JAX 48.5 14.5
MIN BAL 50.5 -6.0
LAC PHI 49.5 1.5
GB KC 48.0 -7.0
ARI SF 45.0 -3.0
TEN LAR 52.5 -7.0
CHI PIT 39.0 -6.5


In [17]:
pylines = spsh.worksheet('PyBettingLines')
pylines.update([week_final.columns.values.tolist()] + week_final.values.tolist())

{'spreadsheetId': '1kPfNdE_HyCEDRKjx24u1917g-xh-vdIAadaPMEMlTrE',
 'updatedCells': 56,
 'updatedColumns': 4,
 'updatedRange': 'PyBettingLines!A1:D14',
 'updatedRows': 14}